In [176]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import time
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestRegressor
#import xgboost as xgb
from sklearn.model_selection import cross_val_score,StratifiedKFold
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.cross_decomposition import PLSRegression
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.feature_selection import f_regression, SelectKBest, VarianceThreshold
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
import datetime as dt
from scipy import stats

df = pd.read_csv("ml_dataset.tsv", sep="\t", index_col=0)


In [16]:
df

,cum_positive,cum_deceased,cum_recovered,curr_positive,hospitalized,intensive_care,iso3,country_name,EUCountry,EUCPMCountry,...,pupils_and_students_enrolled_by_sex_age_and_nuts2.tsv,real_growth_rate_of_regional_gross_value_added_GVA_at_basic_prices_by_nuts2.tsv,stock_of_vehicles_by_category_and_nuts2.tsv,students_enrolled_in_tertiary_education_by_education_level_programme_orientation_sex_and_nuts2.tsv,unemployment_rate_nuts2.tsv,utilised_agricultural_area.tsv,young_people_neither_in_employment_nor_in_education_and_training_by_sex_NEET_RATE_nuts2.tsv,cum_positive_density,cum_deceased_density,cum_recovered_density
ITC4,852193.0,33823.0,808143.0,10227.0,237.0,26.0,ITA,Italy,True,True,...,494636.0,106.4,6967861.0,328322.0,5.0,922320.0,15.7,0.084985,0.003373,0.080592
ITH3,435954.0,11641.0,412386.0,11927.0,119.0,14.0,ITA,Italy,True,True,...,235939.0,105.3,3621253.0,120980.0,5.8,796250.0,12.6,0.089351,0.002386,0.084520
ITH5,393632.0,13282.0,373860.0,6490.0,211.0,14.0,ITA,Italy,True,True,...,212847.0,106.3,3345612.0,169485.0,5.7,1034350.0,13.2,0.088177,0.002975,0.083748
ITH1,73775.0,1183.0,72252.0,340.0,10.0,0.0,ITA,Italy,True,True,...,27774.0,107.8,570247.0,4735.0,3.8,227900.0,10.4,0.138507,0.002221,0.135648
ITG1,242059.0,6043.0,226021.0,9995.0,298.0,30.0,ITA,Italy,True,True,...,237664.0,99.8,3787735.0,119324.0,17.9,1326110.0,29.3,0.049650,0.001240,0.046361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SE11,257954.0,0.0,0.0,257954.0,0.0,0.0,SWE,Sweden,True,True,...,213236.0,111.7,1088426.0,90684.0,7.6,92000.0,6.7,0.108517,0.000000,0.000000
UKD4,53003.0,0.0,0.0,53003.0,0.0,0.0,GBR,United Kingdom,False,True,...,NaN,NaN,798581.0,NaN,3.7,218720.0,13.7,0.035446,0.000000,0.000000
AL02,64450.0,1249.0,63009.0,192.0,11.0,0.0,ALB,Albania,False,False,...,NaN,119.0,NaN,NaN,NaN,NaN,NaN,0.054793,0.001062,0.053568
PT20,6640.0,34.0,0.0,6606.0,0.0,0.0,PRT,Portugal,True,True,...,16669.0,108.5,0.0,2673.0,6.1,118590.0,17.2,0.027348,0.000140,0.000000


In [205]:
X = df.iloc[:,11:-3 ]
y = df["cum_positive_density"] *100


In [178]:
def percent_avbl_nation(nation, column, dataframe):
    not_nans_for_region = dataframe[(dataframe[column].isna() == False) & 
                            dataframe["GEO (Nation Codes)"].apply(lambda x: nation == x)]
    full_for_region = dataframe[dataframe["GEO (Nation Codes)"].apply(lambda x: nation == x)]
    
    return len(not_nans_for_region) / len(full_for_region)
def percent_nans(column, dataframe):
    return len(dataframe[dataframe[column].isna()])/len(dataframe)

for column in X.columns:
    print(column, " ", percent_nans(column, X))


air_passengers.tsv   0.22727272727272727
available_hospital_beds_nuts2.tsv   0.1590909090909091
causes_of_death_crude_death_rate_3year_average_by_nuts2.tsv   0.022727272727272728
compensation_of_employees_by_nuts2.tsv   0.12121212121212122
deaths.tsv   0.0
early_leavers_from_education_and_training_by_sex_percentage_nuts2.tsv   0.07575757575757576
employment_thousand_hours_worked_nuts2.tsv   0.17424242424242425
farm_labour_force.tsv   0.06060606060606061
health_personnel_by_nuts2.tsv   0.11363636363636363
hospital_discharges_resp_diseases_j00_to_j99_nuts2.tsv   0.3106060606060606
life_expectancy.tsv   0.0
longterm_care_beds_per_hundred_thousand_nuts2.tsv   0.29545454545454547
nama_10r_2gdp.tsv   0.09848484848484848
participation_in_education_and_training.tsv   0.022727272727272728
population_nuts2.tsv   0.0
pop_density.tsv   0.0
pupils_and_students_enrolled_by_sex_age_and_nuts2.tsv   0.06818181818181818
real_growth_rate_of_regional_gross_value_added_GVA_at_basic_prices_by_nuts2.tsv   0.

In [206]:
X.fillna(X.mean(axis=0), inplace=True)
X = (X - X.mean(axis=0))/ X.std(axis=0)
#for column in X.columns:
#    X[column] = (X[column] - X[column].mean())/ X[column].std()

In [208]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, shuffle=True)#, stratify=mean_analysis_df["GEO (Nation Codes)"])
std = StandardScaler()
pipeline = Pipeline( steps= [#("std", StandardScaler()),
                                          ("lm", RandomForestRegressor())#RandomForestRegressor(max_depth=5, n_estimators=200))#xgb.XGBRegressor(n_estimators=500)),
                                         ])
#lm = LinearRegression()
pipeline.fit(X_train, y_train)

#cross_v
y_train_preds = pipeline.predict(X_train)
y_preds = pipeline.predict(X_test)
print(r2_score(y_train, y_train_preds))
print(r2_score(y_test, y_preds))

0.9081920518494564
0.21499027168878893


In [212]:

N_SPLITS = 10
#skf = StratifiedKFold(n_splits=N_SPLITS)
#skf.get_n_splits(X, mean_analysis_df["GEO (Nation Codes)"])
kf = KFold(n_splits=N_SPLITS)
kf.get_n_splits(X)


X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.40, shuffle=True)#, stratify=mean_analysis_df["GEO (Nation Codes)"])

tot_train_rscore = 0
tot_valid_rscore = 0
tot_test_rscore = 0
for train_index, valid_index in kf.split(X_train_full):
    
    #print("TRAIN:", train_index, "TEST:", valid_index)
    X_train, X_valid = X_train_full.iloc[train_index], X_train_full.iloc[valid_index]
    y_train, y_valid = y_train_full.iloc[train_index], y_train_full.iloc[valid_index]
    
    
    pipeline = Pipeline( steps= [#("std", StandardScaler()),
                                          ("lm",RandomForestRegressor())#xgb.XGBRegressor(n_estimators=200, max_depth=5)),
                                             ])
    pipeline.fit(X_train, y_train)

    #cross_v
    y_train_preds = pipeline.predict(X_train)
    y_valid_preds = pipeline.predict(X_valid)
    y_test_preds = pipeline.predict(X_test)
    #print(r2_score(y_train, y_train_preds))
    #print(r2_score(y_valid, y_preds))
    tot_train_rscore += r2_score(y_train, y_train_preds)
    tot_valid_rscore += r2_score(y_valid, y_valid_preds)
    tot_test_rscore += r2_score(y_test, y_test_preds)
print("Train: ", tot_train_rscore/N_SPLITS) 
print("Valid: ", tot_valid_rscore/N_SPLITS)
print("Test: ", tot_test_rscore/N_SPLITS)

trained_model = pipeline["lm"]

for feature, importance in sorted(zip(X.columns, trained_model.feature_importances_), key=lambda x: x[1], reverse=True ):
    print(feature, " ", importance)
    

Train:  0.9067373104929335
Valid:  0.322359399400553
Test:  0.17678673538039646
pop_density.tsv   0.23799177373458588
life_expectancy.tsv   0.14894440009172308
participation_in_education_and_training.tsv   0.12407688710196636
early_leavers_from_education_and_training_by_sex_percentage_nuts2.tsv   0.08119118830724135
young_people_neither_in_employment_nor_in_education_and_training_by_sex_NEET_RATE_nuts2.tsv   0.044393852333162724
available_hospital_beds_nuts2.tsv   0.04328497441963017
longterm_care_beds_per_hundred_thousand_nuts2.tsv   0.04262526210599536
causes_of_death_crude_death_rate_3year_average_by_nuts2.tsv   0.03313934147687517
real_growth_rate_of_regional_gross_value_added_GVA_at_basic_prices_by_nuts2.tsv   0.026778932615467682
stock_of_vehicles_by_category_and_nuts2.tsv   0.02662647635067118
health_personnel_by_nuts2.tsv   0.025827158399467916
unemployment_rate_nuts2.tsv   0.020649606937690343
employment_thousand_hours_worked_nuts2.tsv   0.02050407941485217
air_passengers.tsv 

In [203]:
trained_model = pipeline["lm"]

for feature, importance in sorted(zip(X.columns, trained_model.feature_importances_), key=lambda x: x[1], reverse=True ):
    print(feature, " ", importance)
    

life_expectancy.tsv   0.24388879311513573
longterm_care_beds_per_hundred_thousand_nuts2.tsv   0.07107519458457147
real_growth_rate_of_regional_gross_value_added_GVA_at_basic_prices_by_nuts2.tsv   0.06377398589743476
pop_density.tsv   0.05502894073240974
participation_in_education_and_training.tsv   0.05367582987509163
pupils_and_students_enrolled_by_sex_age_and_nuts2.tsv   0.04938311254391074
causes_of_death_crude_death_rate_3year_average_by_nuts2.tsv   0.048964871828883025
unemployment_rate_nuts2.tsv   0.04578722914921249
early_leavers_from_education_and_training_by_sex_percentage_nuts2.tsv   0.042861015083985765
young_people_neither_in_employment_nor_in_education_and_training_by_sex_NEET_RATE_nuts2.tsv   0.04240026476385794
employment_thousand_hours_worked_nuts2.tsv   0.035909886459240196
hospital_discharges_resp_diseases_j00_to_j99_nuts2.tsv   0.031865365926066314
students_enrolled_in_tertiary_education_by_education_level_programme_orientation_sex_and_nuts2.tsv   0.03018224986398452

In [222]:
out_df = pd.DataFrame()

#if includes also categorical
#out_df[df.columns[:11]]= df.iloc[:,:11 ]
out_df[df.columns[-3:]]= df.iloc[:,-3: ]
#out_df[df.columns[:6]]= df.iloc[:,:6 ]

out_df[X.columns] = X


In [223]:
out_df

,cum_positive_density,cum_deceased_density,cum_recovered_density,air_passengers.tsv,available_hospital_beds_nuts2.tsv,causes_of_death_crude_death_rate_3year_average_by_nuts2.tsv,compensation_of_employees_by_nuts2.tsv,deaths.tsv,early_leavers_from_education_and_training_by_sex_percentage_nuts2.tsv,employment_thousand_hours_worked_nuts2.tsv,...,participation_in_education_and_training.tsv,population_nuts2.tsv,pop_density.tsv,pupils_and_students_enrolled_by_sex_age_and_nuts2.tsv,real_growth_rate_of_regional_gross_value_added_GVA_at_basic_prices_by_nuts2.tsv,stock_of_vehicles_by_category_and_nuts2.tsv,students_enrolled_in_tertiary_education_by_education_level_programme_orientation_sex_and_nuts2.tsv,unemployment_rate_nuts2.tsv,utilised_agricultural_area.tsv,young_people_neither_in_employment_nor_in_education_and_training_by_sex_NEET_RATE_nuts2.tsv
ITC4,0.084985,0.003373,0.080592,3.117492,-0.411887,-0.161293,5.526463e+00,5.101050,0.168025,5.544048e+00,...,-0.612886,5.013451,0.150741,4.068790,-4.821979e-01,5.327166,3.542032,-5.342815e-01,0.352790,0.805419
ITH3,0.089351,0.002386,0.084520,0.723851,-0.537448,-0.106162,1.700606e+00,1.945886,-0.101699,2.082955e+00,...,-0.650280,1.905360,-0.046225,1.394922,-6.780014e-01,2.290470,0.711346,-3.672777e-01,0.200243,0.257628
ITH5,0.088177,0.002975,0.083748,0.061022,-0.289413,0.410591,1.660751e+00,2.030752,-0.332892,1.851420e+00,...,-0.450848,1.654819,-0.148806,1.156245,-4.999982e-01,2.040354,1.373549,-3.881532e-01,0.488348,0.363652
ITH1,0.138507,0.002221,0.135648,-0.708574,-0.415148,-0.847903,-5.828515e-01,-0.799340,0.611144,-7.122219e-01,...,-0.712602,-0.718582,-0.314908,-0.756653,-2.329935e-01,-0.477999,-0.875661,-7.847871e-01,-0.487469,-0.131128
ITG1,0.049650,0.001240,0.046361,0.706537,-0.811282,0.032955,4.845890e-01,2.162374,1.612977,9.665894e-01,...,-1.011750,1.903040,-0.161357,1.412751,-1.657019e+00,2.441535,0.688738,2.158654e+00,0.841382,3.208635
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SE11,0.108517,0.000000,0.000000,1.471688,-1.264663,-1.473778,1.778623e+00,-0.125232,-0.698946,5.339605e-01,...,2.154233,0.394890,0.055972,1.160265,4.612190e-01,-0.007805,0.297737,8.480658e-03,-0.651910,-0.784944
UKD4,0.035446,0.000000,0.000000,-0.710290,0.000000,0.230662,1.500936e-16,-0.079464,0.148759,-3.760023e-16,...,0.097590,-0.137423,0.214390,0.000000,-2.529577e-15,-0.270809,0.000000,-8.056625e-01,-0.498577,0.452005
AL02,0.054793,0.001062,0.053568,0.000000,0.000000,0.000000,1.500936e-16,-0.614044,0.000000,-3.760023e-16,...,0.000000,-0.330048,-0.104880,0.000000,1.760642e+00,0.000000,0.000000,1.854114e-16,0.000000,0.000000
PT20,0.027348,0.000140,0.000000,-0.490283,1.105906,-0.335791,-9.108718e-01,-0.934174,3.077196,-9.798849e-01,...,-0.874641,-0.893561,-0.272518,-0.871433,-1.083912e-01,-0.995438,-0.903812,-3.046513e-01,-0.619736,1.070480


In [225]:
out_df.to_csv("eurostat_db.csv")